# Imports

In [360]:
# time management
import datetime

# data import, handling, analysis
import pandas
import seaborn
import os

# data visualisation
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# report production
from reportlab.pdfgen import canvas 
from reportlab.pdfbase.ttfonts import TTFont 
from reportlab.pdfbase import pdfmetrics 
from reportlab.lib import colors 
from reportlab.lib.pagesizes import A4


# Load Data

In [361]:
data = pandas.read_csv('data.csv')
data["Date"] = pandas.to_datetime(data["Date"])
data.set_index("Date", inplace = True)
data.sort_index()

/tmp/ipykernel_16179/1893691555.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data["Date"] = pandas.to_datetime(data["Date"])


,Distance,PetrolFilled(Litres),PetrolType,TotalCost
Date,,,,
2024-11-24,372.6,29.18,ULP98,52.20
2024-12-01,320.3,25.86,ULP98,46.26
2024-12-15,404.2,30.71,ULP98,56.97
2024-12-21,383.1,29.59,ULP98,54.30
2025-01-11,565.7,44.24,ULP98,83.83
2025-01-19,498.3,37.59,ULP98,69.73
2025-01-26,304.8,23.26,ULP98,44.08
2025-02-02,221.0,18.27,ULP98,33.89
2025-02-09,373.3,28.30,ULP98,52.50


## Data Aggregation

In [362]:
data["PetrolPrice(PerLitre)"] = data["TotalCost"]/data["PetrolFilled(Litres)"]
data["Kilometrage(L/100km)"] = (data["PetrolFilled(Litres)"]/data["Distance"])*100
data["CostPerKilometre"] = data["TotalCost"]/data["Distance"]

# Idea

Ideas to include:

- Average Monthly Numeric Statistics 
  - Cost
  - Distance Travelled
  - Fuel Efficiency (L/100)
  - Petrol Filled (Litres)
  - Cost per Kilometre
Also Calculate for YTD and compare


- Expected Statistics for the year
  - expected total distance travelled
  - expected total fuel usage (Litres)
  - expected total cost
  - expected environmental impact (estimate)???



- Visualisation of Statistics
  - Cost of Petrol Over Time
  - Proportion of each petrol type used (ULP91 - ULP98)
  - Fuel Usage Over Time
  - Weekly Cost Distribution
  - Cost vs. Distance
  - Fuel Efficiency Over Time
Also Calculate for YTD and compare where appropriate



- output specified report as a PDF
  - DATE PREPARED
  - WATERMARK ([Zec-Wicks 2024](https://zecwicks.com))
  - Concise introduction paragraph
  - Input specifications
    - date range
    - resulting entries
    - etc.
  - Statistics / Data / Visualations within various sections.
  - Conclusion? 


THE DATA IN THE REPORT SHOULD BE SPLIT INTO CURRENT MONTH (SAME DAY FROM PREVIOUS MONTH UNTIL NOW), AND YEAR TO DATE YTD

# Numeric Statistic Calculation

In [363]:
def numericStatistics(data, startDate = None, endDate = None, function = "sum", monthly = False):
    function = function.lower()

    startDate = data.index.min() if startDate is None else pandas.to_datetime(startDate, dayfirst=True)
    endDate = data.index.max() if endDate is None else pandas.to_datetime(endDate, dayfirst=True)

        
    tmp = data[(startDate <= data.index) & (data.index <= endDate)]

    # Calculate time differences between entries
    tmp["DaysSinceLast"] = tmp.index.to_series().diff().dt.days.fillna(1)

    if monthly:
        tmp = tmp.groupby(tmp.index.to_period("M"))

    if function == "sum":
        return tmp[["Distance", "PetrolFilled(Litres)", "TotalCost"]].sum()
    
    elif function == "average":
        result = tmp[["Distance", "PetrolFilled(Litres)", "TotalCost", "PetrolPrice(PerLitre)", "Kilometrage(L/100km)", "CostPerKilometre"]].mean()

        # Compute the average time between entries (only for non-monthly aggregation)
        if not monthly:
            if len(tmp.index) > 1:  # Ensure there are at least two entries
                averageTimeDifferences = tmp.index.to_series().diff().dt.total_seconds().mean()  # Average Time differences in seconds
                result["AverageTimeBetweenEntries(seconds)"] = pandas.to_timedelta(averageTimeDifferences, unit='s')
            else:
                result["AverageTimeBetweenEntries(seconds)"] = None  # Not enough data points to compute

        return result
    
    elif function == "mode":
        if monthly:
            # Use 'agg' with a lambda function for grouped mode
            return tmp[["PetrolType"]].agg(lambda group: group.mode())
        else:
            # Directly compute mode for non-grouped data
            return tmp[["PetrolType"]].mode()

    else:
        return "Invalid function"


# Data Visualisations

In [ ]:
def dataVisualisations(data):
    figPaths = {}
    
    # Ensure data is sorted by date
    data = data.sort_index()

    # Calculate the number of days between entries, handle first entry separately
    data["DaysSinceLastEntry"] = data.index.to_series().diff().dt.days.fillna(7)

    # Normalise Total Cost & Distance by the number of days (applied once)
    data["NormalisedCost"] = data["TotalCost"] / data["DaysSinceLastEntry"]
    data["NormalisedDistance"] = data["Distance"] / data["DaysSinceLastEntry"]

    # Resample to daily values, using the mean to fill gaps, and handle missing values
    dailyData = data.resample("D").sum()

    # Replace 0s with NaN in 'NormalisedCost' and 'NormalisedDistance'
    dailyData["NormalisedCost"].replace(0, float('nan'), inplace=True)
    dailyData["NormalisedDistance"].replace(0, float('nan'), inplace=True)
    dailyData["Kilometrage(L/100km)"].replace(0, float('nan'), inplace=True)

    dailyData["NormalisedCost"] = dailyData["NormalisedCost"].bfill()
    dailyData["NormalisedDistance"] = dailyData["NormalisedDistance"].bfill()
    dailyData["Kilometrage(L/100km)"] = dailyData["Kilometrage(L/100km)"].bfill()

    # --- Graph 1: Normalised Daily Distance (Dynamic Rolling Avg) ---
    plt.figure(figsize=(8, 4))
    # Apply dynamic rolling average based on the number of days since last entry for distance
    # dailyData["RollingDistance"] = dailyData["Distance"].rolling(window=dailyData["DaysSinceLastEntry"], min_periods=1).mean()
    dailyData["NormalisedDistance"].plot(title='Daily Daily Distance (Dynamic Rolling Avg)', color='blue')
    plt.xlabel('Date')
    plt.xticks(rotation=45)
    plt.ylabel('Distance (KM/day)')
    figPaths['daily_distance'] = "./outputs/daily_distance.png"
    plt.savefig(figPaths['daily_distance'], bbox_inches='tight')
    plt.close()

    # --- Graph 2: Normalised Rolling Cost (Dynamic Rolling Avg) ---
    plt.figure(figsize=(8, 4))
    # Apply dynamic rolling average based on the number of days since last entry for cost
    # dailyData["RollingCost"] = dailyData["TotalCost"].rolling(window=dailyData["DaysSinceLastEntry"], min_periods=1).mean()
    dailyData["NormalisedCost"].plot(title='Daily Fuel Cost (Normalised)', color='green')
    plt.xlabel('Date')
    plt.xticks(rotation=45)
    plt.ylabel('Cost per Day ($)')
    figPaths['rolling_money_spent'] = "./outputs/rolling_money_spent.png"
    plt.savefig(figPaths['rolling_money_spent'], bbox_inches='tight')
    plt.close()

    # --- Graph 3: Fuel Economy Over Time ---
    plt.figure(figsize=(8, 4))
    dailyData["Kilometrage(L/100km)"].plot(title='Daily Kilometerage (L/100) (Normalised)', color='red')
    plt.xlabel('Date')
    plt.xticks(rotation=45)
    plt.ylabel('Fuel Economy (L/100KM)')
    figPaths['fuel_economy'] = "./outputs/fuel_economy.png"
    plt.savefig(figPaths['fuel_economy'], bbox_inches='tight')
    plt.close()

    return figPaths


# Predictive Data Modelling

In [365]:
def predictiveModelling(data, predictionStartDate):
    fileOutputs = []

    

    return fileOutputs

# Report Generation

In [366]:
pastMonthEnd = datetime.datetime.now()
pastMonthStart = pastMonthEnd - pandas.to_timedelta("28day")

currentMonthTotals = numericStatistics(data,pastMonthStart,pastMonthEnd,"sum",False)
currentMonthAverages = numericStatistics(data,pastMonthStart,pastMonthEnd,"average",False)

# Data this calendar year
calendarYearStart = datetime.datetime.now().replace(month=1, day=1, hour=0, minute=0, second=0)
calendarYearEnd = datetime.datetime.now().replace(month=12, day=31, hour=23, minute=59, second=59)
calendarYearTotals = numericStatistics(data,calendarYearStart,calendarYearEnd,"sum",False)
calendarYearAverages = numericStatistics(data,calendarYearStart,calendarYearEnd,"average",False)
# predictiveModelling

# Data all time
allTimeSums = numericStatistics(data,function="sum")
allTimeAverages = numericStatistics(data,function="average")

allTimeMonthlySums = numericStatistics(data,function="sum",monthly=True)
allTimeMonthlyAverages = numericStatistics(data,function="average",monthly=True)

/tmp/ipykernel_16179/1712809719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["DaysSinceLast"] = tmp.index.to_series().diff().dt.days.fillna(1)
/tmp/ipykernel_16179/1712809719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["DaysSinceLast"] = tmp.index.to_series().diff().dt.days.fillna(1)
/tmp/ipykernel_16179/1712809719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [ ]:
def produceReport(data, reportTitle):
    pdfFilename = './outputs/fuel_consumption_report.pdf'
    c = canvas.Canvas(pdfFilename, pagesize=A4)
    width, height = A4
    y = height - 50  # Initial position
    lineSpacing = 20

    def drawLine(text):
        nonlocal y
        if y < 100:
            c.showPage()
            y = height - 50
        c.drawString(50, y, text)
        y -= lineSpacing
    
    # Current month Report
    c.setFont("Helvetica-Bold", 18)
    drawLine(f"{datetime.datetime.now().strftime('%B')} {datetime.datetime.now().year} Fuel Consumption Report")

    # Current Month Report Content
    c.setFont("Helvetica", 12)
    drawLine(f"Total Cost: ${currentMonthTotals['TotalCost']:.2f}")
    drawLine(f"Total Distance: {currentMonthTotals['Distance']:.2f} KM")
    drawLine(f"Total Petrol Consumption: {currentMonthTotals['PetrolFilled(Litres)']:.2f} L")
    drawLine(f"Fuel Economy: {round(currentMonthAverages['Kilometrage(L/100km)'],2)}L / 100 KM")
    drawLine(f"Cost Per Kilometre: ${round(currentMonthAverages['CostPerKilometre'], 2)}")
    drawLine("")

    # Current Year Report
    c.setFont("Helvetica-Bold", 18)
    drawLine(f"{datetime.datetime.now().year} Fuel Consumption Report")

    # Current Year Report Content
    c.setFont("Helvetica", 12)
    drawLine(f"Total Cost: ${calendarYearTotals['TotalCost']:.2f}")
    drawLine(f"Total Distance: {calendarYearTotals['Distance']:.2f} KM")
    drawLine(f"Total Petrol Consumption: {calendarYearTotals['PetrolFilled(Litres)']:.2f} L")
    drawLine(f"Fuel Economy: {round(calendarYearAverages['Kilometrage(L/100km)'],2)}L / 100 KM")
    drawLine(f"Cost Per Kilometre: ${round(calendarYearAverages['CostPerKilometre'], 2)}")
    drawLine("")

    # All Time Report
    c.setFont("Helvetica-Bold", 18)
    drawLine("All-Time Fuel Consumption Report")
    
    # All Time Report Content
    c.setFont("Helvetica", 12)
    drawLine(f"Total Cost: ${allTimeSums['TotalCost']:.2f}")
    drawLine(f"Total Distance: {allTimeSums['Distance']:.2f} KM")
    drawLine(f"Total Petrol Consumption: {allTimeSums['PetrolFilled(Litres)']:.2f} L")
    drawLine(f"Fuel Economy: {round(allTimeAverages['Kilometrage(L/100km)'],2)}L / 100 KM")
    drawLine(f"Cost Per Kilometre: ${round(allTimeAverages['CostPerKilometre'], 2)}")

    c.showPage()
    y = height - 100  # Initial position

    # Load and Add Charts
    figPaths = dataVisualisations(data)
    
    for figPath in figPaths.values():
        if os.path.exists(figPath):
            if y < 200:
                c.showPage()
                y = height - 50
            c.drawImage(figPath, 50, y - 150, width=400, height=200)
            y -= 230
    s
    c.save()
    print(f"Report saved to {pdfFilename}")
    return pdfFilename

# Run visualisations and generate the report
produceReport(data, "Fuel Consumption Report")

/tmp/ipykernel_16179/877058062.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dailyData["NormalisedCost"].replace(0, float('nan'), inplace=True)
/tmp/ipykernel_16179/877058062.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

,Distance,PetrolFilled(Litres),PetrolType,TotalCost,PetrolPrice(PerLitre),Kilometrage(L/100km),CostPerKilometre,DaysSinceLastEntry,NormalisedCost,NormalisedDistance
Date,,,,,,,,,,
2024-11-24,372.6,29.18,ULP98,52.20,1.788897,7.831455,0.140097,7.0,7.457143,53.228571
2024-12-01,320.3,25.86,ULP98,46.26,1.788863,8.073681,0.144427,7.0,6.608571,45.757143
2024-12-15,404.2,30.71,ULP98,56.97,1.855096,7.597724,0.140945,14.0,4.069286,28.871429
2024-12-21,383.1,29.59,ULP98,54.30,1.835079,7.723832,0.141738,6.0,9.050000,63.850000
2025-01-11,565.7,44.24,ULP98,83.83,1.894892,7.820400,0.148188,21.0,3.991905,26.938095
2025-01-19,498.3,37.59,ULP98,69.73,1.855015,7.543648,0.139936,8.0,8.716250,62.287500
2025-01-26,304.8,23.26,ULP98,44.08,1.895099,7.631234,0.144619,7.0,6.297143,43.542857
2025-02-02,221.0,18.27,ULP98,33.89,1.854953,8.266968,0.153348,7.0,4.841429,31.571429
2025-02-09,373.3,28.30,ULP98,52.50,1.855124,7.581034,0.140638,7.0,7.500000,53.328571


,Distance,PetrolFilled(Litres),PetrolType,TotalCost,PetrolPrice(PerLitre),Kilometrage(L/100km),CostPerKilometre,DaysSinceLastEntry,NormalisedCost,NormalisedDistance
Date,,,,,,,,,,
2024-11-24,372.6,29.18,ULP98,52.20,1.788897,7.831455,0.140097,7.0,7.457143,53.228571
2024-11-25,0.0,0.00,0,0.00,0.000000,8.073681,0.000000,0.0,6.608571,45.757143
2024-11-26,0.0,0.00,0,0.00,0.000000,8.073681,0.000000,0.0,6.608571,45.757143
2024-11-27,0.0,0.00,0,0.00,0.000000,8.073681,0.000000,0.0,6.608571,45.757143
2024-11-28,0.0,0.00,0,0.00,0.000000,8.073681,0.000000,0.0,6.608571,45.757143
...,...,...,...,...,...,...,...,...,...,...
2025-03-26,0.0,0.00,0,0.00,0.000000,8.770227,0.000000,0.0,4.864286,30.900000
2025-03-27,0.0,0.00,0,0.00,0.000000,8.770227,0.000000,0.0,4.864286,30.900000
2025-03-28,0.0,0.00,0,0.00,0.000000,8.770227,0.000000,0.0,4.864286,30.900000


Report saved to ./outputs/fuel_consumption_report.pdf


'./outputs/fuel_consumption_report.pdf'

In [368]:
def produceReport(data):

    # Data from last 28 days (4 weeks)
    pastMonthEnd = datetime.datetime.now()
    pastMonthStart = pastMonthEnd - pandas.to_timedelta("28day")
    
    currentMonthTotals = numericStatistics(data,pastMonthStart,pastMonthEnd,"sum",False)
    currentMonthAverages = numericStatistics(data,pastMonthStart,pastMonthEnd,"average",False)


    # Data this calendar year
    calendarYearStart = datetime.datetime.now().replace(month=1, day=1, hour=0, minute=0, second=0)
    calendarYearEnd = datetime.datetime.now().replace(month=12, day=31, hour=23, minute=59, second=59)

    calendarYearTotals = numericStatistics(data,calendarYearStart,calendarYearEnd,"sum",False)

    # predictiveModelling
    

    # Data all time
    allTimeSums = numericStatistics(data,function="sum")
    allTimeAverages = numericStatistics(data,function="average")

    allTimeMonthlySums = numericStatistics(data,function="sum",monthly=True)
    allTimeMonthlyAverages = numericStatistics(data,function="average",monthly=True)

    # dataVisualisationFiles = dataVisualisations()

    # put results into PDF form using reportlab

    # print as a placeholder
    print(f"{datetime.datetime.now().strftime('%B')}, {datetime.datetime.now().year}")
    print("Fuel Consumption Report")

    print(f"\n\n{datetime.datetime.now().strftime('%B')}, {datetime.datetime.now().year} Report")
    print(f"{datetime.datetime.now().strftime('%B')} Totals")
    print(f"\tCurrent Total Cost: ${currentMonthTotals['TotalCost']}")
    print(f"\tCurrent Total Distance Travelled: {round(currentMonthTotals['Distance'],2)} KM")
    print(f"\tCurrent Total Petrol Consumption: {currentMonthTotals['PetrolFilled(Litres)']} L")

    print(f"\n{datetime.datetime.now().strftime('%B')} Average")
    print(f"\tFuel Economy: {round(currentMonthAverages['Kilometrage(L/100km)'],2)}L / 100 KM")
    print(f"\tCost Per Kilometre: ${round(currentMonthAverages['CostPerKilometre'], 2)}")
    print(f"\tTime Between Refills: {currentMonthAverages['AverageTimeBetweenEntries(seconds)'].days} Days")

    print("\n\nAll Time Report")
    print("\tOverall Totals:")
    print(f"\t\tTotal Cost: ${allTimeSums['TotalCost']}")
    print(f"\t\tTotal Distance: {allTimeSums['Distance']} KM")
    print(f"\t\tTotal Petrol Consumption: {allTimeSums['PetrolFilled(Litres)']} L")
    print("\tOverall Averages:")
    print(f"\t\tAverage Cost: ${round(allTimeAverages['TotalCost'],2)}")
    print(f"\t\tAverage Distance: {round(allTimeAverages['Distance'],2)} KM")
    print(f"\t\tAverage Petrol Consumption: {round(allTimeAverages['PetrolFilled(Litres)'],2)} L")
    print(f"\t\tAverage Fuel Economy: {round(allTimeAverages['Kilometrage(L/100km)'],2)}L / 100KM")

    print(f"\n\nYear ({datetime.datetime.now().year}) to Date Report")
    print("\tYear to Date Totals:")
    print(f"\t\tTotal Cost: ${calendarYearTotals['TotalCost']}")
    print(f"\t\tTotal Distance: {calendarYearTotals['Distance']} KM")
    print(f"\t\tTotal Petrol Consumption: {round(calendarYearTotals['PetrolFilled(Litres)'],2)} L")
    
    return

produceReport(data)

March, 2025
Fuel Consumption Report


March, 2025 Report
March Totals
	Current Total Cost: $108.99
	Current Total Distance Travelled: 664.0 KM
	Current Total Petrol Consumption: 59.239999999999995 L

March Average
	Fuel Economy: 8.89L / 100 KM
	Cost Per Kilometre: $0.16
	Time Between Refills: 7 Days


All Time Report
	Overall Totals:
		Total Cost: $724.3399999999998
		Total Distance: 4919.500000000001 KM
		Total Petrol Consumption: 392.5 L
	Overall Averages:
		Average Cost: $48.29
		Average Distance: 327.97 KM
		Average Petrol Consumption: 26.17 L
		Average Fuel Economy: 8.12L / 100KM


Year (2025) to Date Report
	Year to Date Totals:
		Total Cost: $514.61
		Total Distance: 3439.3 KM
		Total Petrol Consumption: 277.16 L


/tmp/ipykernel_16179/1712809719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["DaysSinceLast"] = tmp.index.to_series().diff().dt.days.fillna(1)
/tmp/ipykernel_16179/1712809719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["DaysSinceLast"] = tmp.index.to_series().diff().dt.days.fillna(1)
/tmp/ipykernel_16179/1712809719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat